In [45]:
import numpy as np 
import pandas as pd 

from __init__ import * 

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, RandomSampler

from models.bert import BERTDataset, BERTClassifier, bert_evaluate
from models.utils import sentiment_score
from transformers import BertTokenizer, BertModel

In [40]:
class args:
    max_seq_length = 128
    dr_rate = 0.3
    hidden_dim = 312
    model_name_or_path = 'huawei-noah/TinyBERT_General_4L_312D'
    learning_rate = 1e-5
    save_path = SAVE_PATH
    batch_size = 512
    num_epochs = 100
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [41]:
test_path = os.path.join(YELP_DIR, 'test.csv')
test = pd.read_csv(test_path, encoding='utf-8-sig')

test.stars = test.stars.apply(sentiment_score)

model = BertModel.from_pretrained(args.model_name_or_path)
tokenizer = BertTokenizer.from_pretrained(args.model_name_or_path)

model = BERTClassifier(args, model)

Some weights of the model checkpoint at huawei-noah/TinyBERT_General_4L_312D were not used when initializing BertModel: ['fit_denses.1.bias', 'fit_denses.2.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'fit_denses.2.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'fit_denses.4.weight', 'fit_denses.3.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'fit_denses.1.weight', 'cls.predictions.bias', 'fit_denses.0.bias', 'fit_denses.4.bias', 'fit_denses.0.weight', 'fit_denses.3.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing

In [42]:
param_path = os.path.join(SAVE_PATH, 'bert_parameters.pt')
model.load_state_dict(torch.load(param_path))

model = model.to(args.device)

In [48]:
testset = BERTDataset(args, test, tokenizer)
test_loader = DataLoader(testset, batch_size=args.batch_size, sampler=RandomSampler)

In [51]:
criterion = nn.BCEWithLogitsLoss().to(args.device)

accs, losses = [], []
for _ in range(5):
    testset = BERTDataset(args, test, tokenizer)
    test_loader = DataLoader(testset, batch_size=args.batch_size, sampler=RandomSampler(testset), num_workers=4)

    acc, loss = bert_evaluate(args, model, test_loader, criterion)
    loss = loss.detach().cpu().item()
    accs.append(acc)
    losses.append(loss)

print(np.mean(accs), np.mean(losses))
print(np.std(accs), np.std(losses))

Evaluating...: 100%|██████████| 472/472 [01:43<00:00,  4.57it/s]

0.8825170840048872 0.0006397495566260213
2.5308764362165446e-05 8.407106744571803e-05


In [57]:
a = list(map(lambda x: x*100, accs))

In [61]:
np.std(a), np.mean(a)

(0.0025308764362169363, 88.25170840048872)